This notebook performs the following tasks:

5.1 [Classify same-day permits](#5.1)


5.2 [Regress non-same-day permits](#5.2)

Kush did tasks 5.1-5.2. 

In [1]:
# Data tools
import pandas as pd
import matplotlib.pyplot as plt

# ML tools
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix

In [2]:
train = pd.read_csv('data_train_clean1.csv')
test = pd.read_csv('data_test_clean1.csv')

# 5.1 Classify same-day permits
<a id='5.1'></a>

In [19]:
def features(data):
    permit_type = np.array(pd.get_dummies(data['Permit Type']))
    existing_constrxn = np.array(pd.get_dummies(train['Existing Construction Type']))
    proposed_constrxn = np.array(pd.get_dummies(train['Proposed Construction Type']))
    existing_use = np.array(pd.get_dummies(train['Existing Use']))
    proposed_use = np.array(pd.get_dummies(train['Proposed Use']))
    plansets = np.array(pd.get_dummies(train['Plansets']))

    existing_units = np.array(train['Existing Units'])
    proposed_units = np.array(train['Proposed Units'])
    fire_only = np.array(train['Fire Only Permit'])
    est_cost = np.array(train['Estimated Cost'])
    retrofit = np.array(train['Voluntary Soft-Story Retrofit'])

    X = np.column_stack((permit_type,
                         existing_constrxn, proposed_constrxn,
                         existing_use, proposed_use,
                         plansets,
                         existing_units, proposed_units,
                         fire_only, est_cost, retrofit))
    
    print(f"Features have shape {X.shape}")
    return X

In [ ]:
X_tr = features(train)
X_te = features(test)

In [33]:
np.savetxt("X_tr.csv", X_tr, delimiter=",")
np.savetxt("X_te.csv", X_te, delimiter=",")

**Imputed data in notebook "5 - GLRM"**

In [ ]:
X_tr_hat = np.loadtxt("X_tr_hat.csv", delimiter=",") # GLRM-imputed data
X_te_hat = np.loadtxt("X_te_hat.csv", delimiter=",") # GLRM-imputed data from training set's W matrix

y_tr = train['Days to issue']
y_te = test['Days to issue']

In [ ]:
def filter_unissued(X, y):
    issued_permits = np.array([i for i,unissued in enumerate(y.isna()) if not unissued])
    X = X[issued_permits,:]
    y = y[issued_permits]
    return X, y

In [ ]:
X_tr_hat, y_tr = filter_unissued(X_tr_hat, y_tr)
X_te_hat, y_te = filter_unissued(X_te_hat, y_te)

print(X_tr_hat.shape)
print(y_tr.shape)

In [ ]:
def binarize_label(y):
    baseline = sum(y == 0)/len(y)
    print(f"Baseline accuracy = {baseline}")
    
    y_bin = y.copy()
    y_bin[y_bin > 0] = 1
    return y_bin

In [ ]:
y_tr_bin = binarize_label(y_tr)
y_te_bin = binarize_label(y_te)

Mini test before running on full training set

In [ ]:
n_sample = 30000
rand_permits = np.random.randint(0, len(y_tr)+1, n_sample)
X_sample = X_tr_hat[rand_permits,:]
y_sample = y_tr_bin[rand_permits,:]

X_sample_tr, X_sample_te, y_sample_tr, y_sample_te = train_test_split(X_sample, y_sample, test_size=0.2)

In [ ]:
def svm(X_tr, X_te, y_tr, y_te):
    clf = SVC(C=1.0, gamma='auto')
    clf.fit(X_tr, y_tr)
    
    train_acc = clf.score(X_tr, y_tr) 
    test_acc  = clf.score(X_te, y_te)
    print(f"Training accuracy = {train_acc}")
    print(f"Test accuracy     = {test_acc}")
    return clf

In [ ]:
clf_sample = svm(X_sample_tr, X_sample_te, y_sample_tr, y_sample_te)

In [ ]:
clf = svm(X_tr_hat, X_te_hat, y_tr_bin, y_te_bin)

# 5.2 Regress non-same-day permits
<a id='5.2'></a>

In [ ]:
def filter_sameday(X, y):
    sameday_bools = (y == 0)
    # nsps = non-same-day permits
    nsps = np.array([i for i,sameday in enumerate(sameday_bools) if not sameday])
    X = X[nsps,:]
    y = y[nsps]
    return X, y

In [ ]:
X_tr_hat_nsp, y_tr_nsp = filter_sameday(X_tr_hat, y_tr)
X_te_hat_nsp, y_te_nsp = filter_sameday(X_te_hat, y_te)

In [ ]:
plt.hist(y_tr_nsp)
plt.xlabel("Days to issue non-same-day permits")

In [ ]:
regr = RandomForestRegressor(max_depth=5)
regr.fit(X_tr_hat_nsp, y_tr_nsp)

In [ ]:
def summarize_acc(X, y):
    R_sq = regr.score(X, y)
    pred = np.maximum(regr.predict(X_tr), 1)
    MSE = np.mean((pred-y)**2)
    RMSE = MSE**0.5
    
    print(f"R^2  = {R_sq}")
    print(f"RMSE = {RMSE}")

    # where is underfitting?
    plt.scatter(pred, y)
    plt.xlabel("Prediction")
    plt.ylabel("Observation")

In [ ]:
summarize_acc(X_tr_hat_nsp, y_tr_nsp)

In [ ]:
summarize_acc(X_te_hat_nsp, y_te_nsp)

Here we determine if the complicated stuff above (added features, GLRM, same-day classifier, RandomForest regressor) was worth the effort:

1. Did it improve on the linear regression? 

2. Is it deployable?

If we wanted to be fancier, we could use a zero-inflation GLM with a negative-binomial distribution, since the response is overdispersed:  

In [ ]:
print(f"Mean days to issue     = {np.mean(y_tr_nsp)}")
print(f"Variance days to issue = {np.var(y_tr_nsp)}")